In [1]:
%matplotlib inline
import IPython.core.display         
# setup output image format (Chrome works best)
IPython.core.display.set_matplotlib_formats("svg")
import matplotlib.pyplot as plt
import matplotlib
from numpy import *
from sklearn import *
from scipy import stats
random.seed(100)
import pandas as pd
import csv
import numpy as np
import os

C:\Users\man\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\man\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
C:\Users\man\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
C:\User

In [2]:
features_name=[
    # player 1 = big blind bets, always action player
    'preflop',
    'flop',
    'turn',
    'river',
    'private_card_1_number', # 0~12
    'private_card_1_suit', # d:0, c:1, h:2, s:3
    'private_card_2_number',
    'private_card_2_suit',
    'public_card_1_number',
    'public_card_1_suit',
    'public_card_2_number',
    'public_card_2_suit',
    'public_card_3_number',
    'public_card_3_suit',
    'public_card_4_number',
    'public_card_4_suit',
    'public_card_5_number',
    'public_card_5_suit',   
    'pot_ratio',
    'action',       # action
]

In [3]:
suit_values = {
    'd':1,
    'c':2,
    'h':3,
    's':4,
}

num_values = {
    '2':1,
    '3':2,
    '4':3,
    '5':4,
    '6':5,
    '7':6,
    '8':7,
    '9':8,
    'T':9,
    'J':10,
    'Q':11,
    'K':12,
    'A':13,
}

action_map = {
    'f':0,
    'c':1,
    'r':2,
}

def cards_array(s):
    a = np.zeros((4,13))
    if (isinstance(s, list)):
        for _s in s:
            a[suit_values[_s[1]]][num_values[_s[0]]] = 1
    else:
        a[suit_values[s[1]]][num_values[s[0]]] = 1
        
    return a

def read_actions(actions):
    action_array = []
    bets = ''
    for ac in actions:
        if ac == 'c' or ac == 'f' or ac == 'r':
            if bets != '':
                action_array.append(int(bets))
                bets = ''
            action_array.append(ac)
        else:
            bets += ac
    return action_array

def read_approach_1(line):
    state, rounds, all_actions, all_cards, bets, names = line.split(':')
    
    # read actions
    actions = all_actions.split('/')
    n_action = len(actions)
    rows = []

    pot = 0
    p2_bet = p1_bet = 0
    for i in range(n_action):
        # init variables
        if i == 0:
            is_p1 = False
            
        row = np.zeros(20)
                
        action_array = read_actions(actions[i])
#         print(action_array)
        
        if len(action_array) == 0:
            break

        if i == 0:
            p2_bet = 100
            p1_bet = 50
        pot_ratio = 0
        c = 0
        l = len(action_array)
        while c < l:
            row = np.zeros(20)
            # preflop, flop, turn, river
            row[i] = 1
            # read cards
            cards = all_cards.split('/')
            private_cards = cards[0]        
            row[4] = num_values[private_cards[0]]
            row[5] = suit_values[private_cards[1]]
            row[6] = num_values[private_cards[2]]
            row[7] = suit_values[private_cards[3]]

            if i > 1:
                row[8] = num_values[cards[1][:2][0]]
                row[9] = suit_values[cards[1][:2][1]]
                row[10] = num_values[cards[1][2:4][0]]
                row[11] = suit_values[cards[1][2:4][1]]
                row[12] = num_values[cards[1][4:6][0]]
                row[13] = suit_values[cards[1][4:6][1]]
                row[18] = 0.5
            if i > 2:
                row[14] = num_values[cards[2][0]]
                row[15] = suit_values[cards[2][1]]
            if i > 3:
                row[16] = num_values[cards[3][0]]
                row[17] = suit_values[cards[3][1]]

#             print(action_array, c)
            
            row[19] = action_map[action_array[c]]
        
            if c == 0:
                is_p1 = c >= 1
            
            if is_p1:
                o_bet = p2_bet
            else:
                o_bet = p1_bet
                
            pot_ratio = o_bet / (pot+p1_bet+p2_bet)
#             print(pot_ratio, p1_bet, p2_bet, pot)
            row[18] = pot_ratio
            
            rows += [row]
                
            if (action_array[c] == 'r'):
                if is_p1:
                    p2_bet = action_array[c+1]
                else:
                    p1_bet = action_array[c+1]
                c += 2    
            elif (action_array[c] == 'c'):
                c += 1
            elif action_array[c] == 'f':
                break
            
            if c>=l:
                pot += 2 * max(p1_bet,p2_bet)
                p1_bet = p2_bet = 0
                
            is_p1 = !is_p1        

        
            
#     print(rows)
    return rows

def position_of_next_dot(s):
    index = 0
    i = s.find('.')
    index = i
    index += s[i+1:].find('.')
    return index + 1

csv_name = 'converted_1_'
# write CSV file
f = open(csv_name + '.csv', 'w')
writer = csv.writer(f)
writer.writerow(features_name)
f.close()

# test_csv_name = 'converted_1_test'
# tf = open(test_csv_name + '.csv', 'w')
# tf_writer = csv.writer(tf)
# tf_writer.writerow(features_name)
# tf.close() 

limit = 1
count = 0
name = ''
n = 0  #number of documents
j = 0

for filename in os.listdir('logs_2pn_2017'):
    if (filename[:position_of_next_dot(filename)] != name):
        name = filename[:position_of_next_dot(filename)]
        count = 0
    count+=1
    if count > limit:
        continue
    if not (filename.startswith("ASHE") and filename.endswith(".log")): 
        continue

    # control number of documents
    n += 1
    if n > 1:
        break
        
    rows = []
    ln = 0
    f = open("logs_2pn_2017\\" + filename, "r")
    print('converting {}...'.format(filename))
    for line in f:
        break;
        # igonre not STATE
        if not line.startswith('STATE'):
            continue

#         ln += 1
#         print(ln)
        rows = read_approach_1(line)

        cf = open(csv_name + '.csv', 'a')
        writer = csv.writer(cf)
        writer.writerows(rows)
        cf.close()

    f.close()
    print('{} Saved to {}.csv'.format(filename, csv_name))


converting ASHE.ElDonatoro.1.0.log...
ASHE.ElDonatoro.1.0.log Saved to converted_1_.csv


In [4]:
# csv_name = 'converted_1'
# # write CSV file
# f = open(csv_name + '.csv', 'w')
# writer = csv.writer(f)
# writer.writerow(features_name)
# writer.writerows(rows)
# f.close()
# print('Saved to {}.csv'.format(csv_name))